In [41]:
import numpy as np 
import pandas as pd 
import random
from collections import Counter
from typing import Tuple
from typing import List
import sys
from matplotlib import pyplot as plt

In [2]:
class colors():
    
    __slots__ = '__possibilities'
    
    def __init__(self):
        self.__possibilities = ['♠', '♦', '♥', '♣'] #['S','H','D','C'] #no need to ensure type
    
    def getPossibilities(self):
        return self.__possibilities

In [3]:
class numbers():
    
    __slots__ = '__possibilities'
    
    def __init__(self):
        self.__possibilities = ['2','3','4','5','6','7','8','9','J','Q','K','A'] #no need to ensure type
    
    def getPossibilities(self):
        return self.__possibilities

In [4]:
class card():
    __slots__ = '__color', '__number'
    
    def __init__(self,aColor, aNumber):
        
        self.__color = aColor
        self.__number = aNumber
        
    def getColor(self):
        return self.__color
    
    def getNumber(self):
        return self.__number
    

In [5]:
class hand():
    __slots__ = '__cards', '__size', '__numbersOnly', '__colorsOnly'
    
    def __init__(self):
        self.__size = 0
        self.__cards = []
        self.__numbersOnly = []
        self.__colorsOnly = []
        
    def addCard(self, aCard:card):
        self.__cards.append(aCard)
        self.__numbersOnly.append(aCard.getNumber())
        self.__colorsOnly.append(aCard.getColor())
        self.__size = self.__size + 1
        
    def getCards(self):
        return self.__cards
    
    def getNumbers(self):
        return self.__numbersOnly
    
    def getColors(self):
        return self.__colorsOnly
    
    def visualizeHand(self):
        myVisu = []
        for card in self.__cards:
            myVisu.append(card.getNumber() + card.getColor())
        print(myVisu)
            

        

In [6]:
class deck():
    __slots__ = '__cards'
        
    def __init__(self):
        self.__cards = []
        for color in colors().getPossibilities():
            for number in numbers().getPossibilities():
                self.__cards.append(card(color, number))
        random.shuffle(self.__cards)
    
    def drawCard(self,*kwargs):
        if len(kwargs) == 1:
            self.__cards.remove(kwargs)
            myCardToReturn = kwargs
        else:
            myCardToReturn = self.__cards.pop(0)
        return myCardToReturn
    
    def getDeck(self):
        return self.__cards

In [7]:
class river():
    __slots__ = '__cards', '__size', '__numbersOnly', '__colorsOnly'
    
    def __init__(self):
        self.__size = 0
        self.__cards = []
        self.__numbersOnly = []
        self.__colorsOnly = []
        
    def addCard(self, aCard:card):
        self.__cards.append(aCard)
        self.__numbersOnly.append(aCard.getNumber())
        self.__colorsOnly.append(aCard.getColor())
        self.__size = self.__size + 1
        
    def getCards(self):
        return self.__cards
    
    def getNumbers(self):
        return self.__numbersOnly
    
    def getColors(self):
        return self.__colorsOnly
    
    def visualizeHand(self):
        myVisu = []
        for card in self.__cards:
            myVisu.append(card.getNumber() + card.getColor())
        print(myVisu)
            


In [8]:
def isFlush(aHand:hand) -> bool:
    return max(Counter(aHand.getColors()).values()) >= 5

def isStraight(aHand:hand) -> Tuple[bool, int]:
    first = True
    countConseq = 0
    maxConSeq = 0
    lastCardSeq = 0
    myIndexedSortedHand = sorted([numbers().getPossibilities().index(x) for x in aHand.getNumbers()])
    for idx in myIndexedSortedHand:
        if not first:
            if idx == myLastIdx+1:
                countConseq = countConseq+1
                lastCardSeq = myLastIdx #to compare between straights
            else:
                countConseq = 0
            if countConseq > maxConSeq :
                maxConSeq = countConseq
        first = False
        myLastIdx = idx
    
    return maxConSeq>=5,int(lastCardSeq)
    
    
def howManySameNumber(aHand:hand)-> Tuple[List,List,List]:
    myCount = Counter(aHand.getNumbers())
    
#     nbFourOfAKind = Counter(myCount.values())[4] #how many four of a kind ..
    cards4 = [numbers().getPossibilities().index(x) for x in [k for (k, v) in myCount.items() if v == 4]]
#     nbThreeOfAKind = Counter(myCount.values())[3]
    cards3 = [numbers().getPossibilities().index(x) for x in [k for (k, v) in myCount.items() if v == 3]]
#     nbPair = Counter(myCount.values())[2]
    cards2 = [numbers().getPossibilities().index(x) for x in [k for (k, v) in myCount.items() if v == 2]]
    
    return cards2,cards3,cards4
    
def singleCard(aHand:hand, idxBest=1): #idxBest starts at 1 for best, 2 for snd Best
    
    return sorted([numbers().getPossibilities().index(x) for x in aHand.getNumbers()])[-idxBest] #get number with highest

In [9]:
def getValue(aHand:hand) -> float:
    myValue = 0
    myStraight, myidx = isStraight(aHand)
    myFlush = isFlush(aHand)
    myRoyal = myidx is len(numbers().getPossibilities())-1
    
    myPairs, myThrees, myFour = howManySameNumber(aHand) 
    
    myValue = myValue + myStraight*myFlush*myRoyal*200200000000 + myStraight*myFlush*10200000000*myidx
   
    if len(myFour):
        myFourValue = 560000000*myFour[0]
    else:
        myFourValue = 0
    
    if len(myThrees)==2 or (len(myThrees)==1 and len(myPairs)):
        myFullHouse= 28000000 * max(myThrees)
    else:
        myFullHouse = 0
    
    
    if len(myPairs)==2:
        myTwoPairs = 70000 * sorted(myPairs)[1]+4000 * sorted(myPairs)[0]
    else:
        myTwoPairs = 0
    
    if len(myThrees)==1:
        myThreeCard = 1400000 * max(myThrees)
    else:
        myThreeCard = 0
        
    if len(myPairs)==1:
        myOnePair = 200 * max(myPairs)
    else:
        myOnePair = 0
    
    myValue = myValue + myFourValue + myFullHouse + 40000*myFlush + 2000*myStraight*myidx
    myValue = myValue + myThreeCard + myTwoPairs + myOnePair 
    
    initCoef = 10
    for i in range(0,7):
        myValue = myValue + singleCard(aHand,i+1)*initCoef
        initCoef = initCoef/20
    
    return myValue
    

In [10]:
a = ['K', 'J', 'A','A'] 
possibilities = ['2','3','4','5','6','7','8','9','J','Q','K','A']

sorted(np.unique(a), key=lambda x: possibilities.index(x))

['J', 'K', 'A']

In [11]:
nbPlayers = 20
myDeck = deck()
mySmallHands = [] #individualHand
myBigHands = [] #Hand with River
for i in range(0,nbPlayers):
    mySmallHands.append(hand())
    myBigHands.append(hand())
myRiver = river()

In [12]:

#draw cards
for drawnCard in range(0,2):
    for player in range(0,nbPlayers):
        myCard = myDeck.drawCard()
        mySmallHands[player].addCard(myCard)
        myBigHands[player].addCard(myCard)
        
#draw cards in the river
for drawnCard in range(0,5):
    myCard = myDeck.drawCard()
    
    myRiver.addCard(myCard)
    for player in range(0,nbPlayers):
        myBigHands[player].addCard(myCard)

In [13]:
for i in range(0,nbPlayers):
    print(getValue(myBigHands[i]))

1294.157753125
115.235253125
114.7050625
1904.7352531249999
284094.105065625
576094.205253125
94.103878125
294.15506562499996
1694.2051906250001
294.102565625
94.130190625
114.706503125
1894.706503125
104.705128125
94.10381249999999
1694.209003125
894.1302531250001
1904.7352531249999
1494.184003125
94.180128125


In [14]:
for i in range(0,nbPlayers):
    print(myBigHands[i].visualizeHand())

['8♣', '8♠', 'J♣', '3♦', 'Q♦', '2♦', '6♥']
None
['K♥', 'A♥', 'J♣', '3♦', 'Q♦', '2♦', '6♥']
None
['2♣', 'A♦', 'J♣', '3♦', 'Q♦', '2♦', '6♥']
None
['K♦', 'Q♥', 'J♣', '3♦', 'Q♦', '2♦', '6♥']
None
['6♦', '3♥', 'J♣', '3♦', 'Q♦', '2♦', '6♥']
None
['6♠', 'J♦', 'J♣', '3♦', 'Q♦', '2♦', '6♥']
None
['5♦', '4♠', 'J♣', '3♦', 'Q♦', '2♦', '6♥']
None
['3♠', '8♥', 'J♣', '3♦', 'Q♦', '2♦', '6♥']
None
['J♥', '5♠', 'J♣', '3♦', 'Q♦', '2♦', '6♥']
None
['3♣', '4♣', 'J♣', '3♦', 'Q♦', '2♦', '6♥']
None
['7♥', '5♥', 'J♣', '3♦', 'Q♦', '2♦', '6♥']
None
['7♠', 'A♣', 'J♣', '3♦', 'Q♦', '2♦', '6♥']
None
['7♣', 'Q♣', 'J♣', '3♦', 'Q♦', '2♦', '6♥']
None
['K♣', '4♥', 'J♣', '3♦', 'Q♦', '2♦', '6♥']
None
['2♠', '5♣', 'J♣', '3♦', 'Q♦', '2♦', '6♥']
None
['9♦', 'J♠', 'J♣', '3♦', 'Q♦', '2♦', '6♥']
None
['7♦', '6♣', 'J♣', '3♦', 'Q♦', '2♦', '6♥']
None
['Q♠', 'K♠', 'J♣', '3♦', 'Q♦', '2♦', '6♥']
None
['9♣', '9♥', 'J♣', '3♦', 'Q♦', '2♦', '6♥']
None
['4♦', '9♠', 'J♣', '3♦', 'Q♦', '2♦', '6♥']
None


In [36]:
class game():
    
    __slots__ = "nbPlayers", "deck", "smallHands", "bigHands", "river", "judgerEnabler", 'winner', 'forcedCards'
    
    def __init__(self, nbOfPlayer : int, forcedCards:bool = False):
        self.judgerEnabler = 0
        self.nbPlayers = nbOfPlayer
        self.deck = deck()
        self.smallHands = [] #individualHand
        self.bigHands = [] #Hand with River
        self.forcedCards = False
        
        for i in range(0,nbPlayers):
            self.smallHands.append(hand())
            self.bigHands.append(hand())
            
        self.river = river()
        self.winner = None
        
        
    def dealCards(self,*kwargs) -> "Nothing":
        if self.forcedCards and len(kwargs)!=2:
            sys.stderr("You need to add 2 cards as argument")
        else:
        #draw cards
            if self.judgerEnabler == 0:
                for player in range(0,self.nbPlayers):
                    for drawnCard in range(0,2):
                        if self.forcedCards and player==0:
                            myCard = self.deck.drawCard(kwargs[drawnCard])
                        else:
                            myCard = self.deck.drawCard()
                            
                        self.smallHands[player].addCard(myCard)
                        self.bigHands[player].addCard(myCard)
            else:
                sys.stderr("Already dealt cards to players")
            self.judgerEnabler = 1
                
    def dealFlop(self) -> "Nothing":
        if self.judgerEnabler == 1:
            for drawnCard in range(0,3):
                myCard = self.deck.drawCard()

                self.river.addCard(myCard)
                for player in range(0,self.nbPlayers):
                    self.bigHands[player].addCard(myCard)
        elif self.judgerEnabler == 0:
            sys.stderr("Not dealt cards to players yet, please do")
        elif self.judgerEnabler > 1:
            sys.stderr("Already did the flop")
        self.judgerEnabler = 2
            
    def dealTurn(self) -> "Nothing":
        if self.judgerEnabler == 2:
            for drawnCard in range(0,1):
                myCard = self.deck.drawCard()

                self.river.addCard(myCard)
                for player in range(0,self.nbPlayers):
                    self.bigHands[player].addCard(myCard)
        elif self.judgerEnabler == 0:
            sys.stderr("Not dealt cards to players yet, please do")
        elif self.judgerEnabler == 1:
            sys.stderr("Flop has not been done, please do things in order")
        elif self.judgerEnabler > 2:
            sys.stderr("Already did the Turn, it is now time for the River")
        self.judgerEnabler = 3
        
    def dealRiver(self) -> "Nothing":
        if self.judgerEnabler == 3:
            for drawnCard in range(0,1):
                myCard = self.deck.drawCard()

                self.river.addCard(myCard)
                for player in range(0,self.nbPlayers):
                    self.bigHands[player].addCard(myCard)
            self._judgeTheGame()
            
        elif self.judgerEnabler == 0:
            sys.stderr("Not dealt cards to players yet, please do")
        elif self.judgerEnabler == 1:
            sys.stderr("Flop has not been done, please do things in order")
        elif self.judgerEnabler == 2:
            sys.stderr("Turn has not been done, please do things in order")
        elif self.judgerEnabler > 3:
            sys.stderr("the Game is Over")
        self.judgerEnabler = 4
        
    def _judgeTheGame(self):
        myValues = []
        for i in range(0,self.nbPlayers):
            myValues.append(getValue(self.bigHands[i]))
        self.winner = myValues.index(max(myValues))
        
    def getWinner(self):
        return self.winner

In [42]:
myWinners = []

myNbOfSim = 1000
myForcedCards = True
myNbOfPlayer = 3

if myForcedCards:
    print("Your are player 0!")
   
myCard1 = card('♦','10')
myCard2 = card('♥','10')

for i in range(0,myNbOfSim):
    
    myGame = game(myNbOfPlayer, forcedCards = myForcedCards)
    myGame.dealCards(myCard1, myCard2)
    myGame.dealFlop()
    myGame.dealTurn()
    myGame.dealRiver()
    myWinners.append(myGame.getWinner())
    
    
    

Your are player 0!


(array([350,   0,   0,   0,   0, 298,   0,   0,   0, 352]),
 array([ 0. ,  0.2,  0.4,  0.6,  0.8,  1. ,  1.2,  1.4,  1.6,  1.8,  2. ]))